# EDA of Movielens 100K Dataset

There are four .csv files
- links.csv
- movies.csv
- ratings.csv
- tags.csv

See README_data.txt for an overview of each file

In [133]:
import os
import glob
import pandas as pd

pd.set_option('display.max_columns', None)

In [134]:
# read in each dataset as a separate dataframe
datasets_list = ['links', 'movies', 'ratings', 'tags']

links, movies, ratings, tags = [(pd.read_csv(f'../data/{dataset}.csv',index_col= None)) for dataset in datasets_list]

### Explore "ratings" dataframe

Things we want to know...
- Average rating given by each user
- Average rating per movie
- Number of movies rated by each user
- Number of ratings per movie
- Average number of reviews per movie (how many users rated the movie)
- Highest rated movie (top 5)
- Lowest rated movie (bottom 5)
- User with most review
- Users with least reviews
- Distribution of ratings


For later review
- Distribution of ratings by genre
- Number of reviews by genre

In [135]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Explore "tags" dataframe

User-generated tags

### Concantenate Dataframes

- concantenate movies df, links df, tags df, and ratings df (on movieId column)

In [136]:
# check for null values on the dataframes using
# series.isnull().any()
# movies df and links df are the same size

movies_df = movies.merge(links, how = 'left', on = 'movieId')


In [137]:
df = ratings.merge(movies_df, how='left', on=  ['movieId'])
df

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,113277,949.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,114369,807.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,114814,629.0
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller,4972582,381288.0
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller,4425200,324552.0
100833,610,168250,5.0,1494273047,Get Out (2017),Horror,5052448,419430.0
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi,3315342,263115.0


### Clean Final Dataframe

- Column names in all lower case
- Split out genres using 
- consider tag column purpose
- Split title column into title and year
- convert release year into datetime or leave as string?
- drop tmdbid, tag timestamp column, ratings timestamp column, drop extra column after title/year split
- drop tags column for now, may not use any text analysis

Order of cleaning
- Concat
- split title column into title and year columns
- one-hot encode genres column
- 


In [138]:
# lowercase all names
df = df.rename(columns={'userId':'userid', 'movieId':'movieid'})

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userid     100836 non-null  int64  
 1   movieid    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   title      100836 non-null  object 
 5   genres     100836 non-null  object 
 6   imdbId     100836 non-null  int64  
 7   tmdbId     100823 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 6.9+ MB


In [140]:
# split title column

df[['title', 'year', 'extra']] = df['title'].str.split(r'[(]([0-9][0-9][0-9][0-9])[)]', n=1, expand=True)
#df.head()

In [141]:
# split out genres of the movies

dummies = df['genres'].str.get_dummies(sep='|')
df = pd.concat([df, dummies], axis=1)

df.columns = df.columns.str.strip('() ').str.replace('-','').str.lower()
df.head()

,userid,movieid,rating,timestamp,title,genres,imdbid,tmdbid,year,extra,no genres listed,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance,113228,15602.0,1995,,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller,113277,949.0,1995,,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),Mystery|Thriller,114369,807.0,1995,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller,114814,629.0,1995,,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [142]:
# check for nulls 
df.isnull().sum()

userid               0
movieid              0
rating               0
timestamp            0
title                0
genres               0
imdbid               0
tmdbid              13
year                18
extra               18
no genres listed     0
action               0
adventure            0
animation            0
children             0
comedy               0
crime                0
documentary          0
drama                0
fantasy              0
filmnoir             0
horror               0
imax                 0
musical              0
mystery              0
romance              0
scifi                0
thriller             0
war                  0
western              0
dtype: int64

In [143]:
null_years = df[df['year'].isnull()]
null_list = null_years['title'].tolist()
null_list

# I will drop these movies and shows from the dataset

['Black Mirror',
 'The Adventures of Sherlock Holmes and Doctor Watson',
 'Death Note: Desu nôto (2006–2007)',
 'Maria Bamford: Old Baby',
 'Generation Iron 2',
 'Ready Player One',
 'Babylon 5',
 'Ready Player One',
 'Nocturnal Animals',
 'Cosmos',
 'Ready Player One',
 'The OA',
 'Hyena Road',
 'Moonlight',
 'Ready Player One',
 'Babylon 5',
 'Paterson',
 'Cosmos']

In [144]:
df = df.dropna(axis=0)

In [145]:
df.isnull().sum()

userid              0
movieid             0
rating              0
timestamp           0
title               0
genres              0
imdbid              0
tmdbid              0
year                0
extra               0
no genres listed    0
action              0
adventure           0
animation           0
children            0
comedy              0
crime               0
documentary         0
drama               0
fantasy             0
filmnoir            0
horror              0
imax                0
musical             0
mystery             0
romance             0
scifi               0
thriller            0
war                 0
western             0
dtype: int64

In [146]:
df.head(5)

,userid,movieid,rating,timestamp,title,genres,imdbid,tmdbid,year,extra,no genres listed,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance,113228,15602.0,1995,,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller,113277,949.0,1995,,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),Mystery|Thriller,114369,807.0,1995,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller,114814,629.0,1995,,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [155]:
# convert year column from object to datetime, ignore nulls if possible
df['year'] = pd.to_datetime(df['year']).dt.year
df.head(5)

,userid,movieid,rating,timestamp,title,genres,imdbid,tmdbid,year,extra,no genres listed,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance,113228,15602.0,1995,,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller,113277,949.0,1995,,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),Mystery|Thriller,114369,807.0,1995,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller,114814,629.0,1995,,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [156]:
df.head(5)

,userid,movieid,rating,timestamp,title,genres,imdbid,tmdbid,year,extra,no genres listed,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,964982703,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men,Comedy|Romance,113228,15602.0,1995,,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Heat,Action|Crime|Thriller,113277,949.0,1995,,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en),Mystery|Thriller,114369,807.0,1995,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,"Usual Suspects, The",Crime|Mystery|Thriller,114814,629.0,1995,,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [157]:
# groupby userid for avg rating by user
#.sort_values(['rating'], axis = 0, ascending=True)

# average rating by user
avg_rate_by_user = df.groupby('userid', as_index=False)['rating'].mean().rename(columns={'rating':'avg_rating_by_user'})

# average rating of each movie
avg_rate_by_movie = df.groupby('movieid', as_index=False)['rating'].mean().rename(columns={'rating':'avg_movie_review_rating'})

# number of movies reviewed by each user
count_reviews_by_user = df.groupby('userid', as_index=False)['movieid'].count().rename(columns={'movieid':'number_of_movies_reviewed'})

# number of reviews per movieId
count_reviews_per_movie = df.groupby('movieid', as_index=False)['userid'].count().rename(columns={'userid':'count_of_reviews'})

# avg number of ratings per movie
avg_count_of_reviews_per_movie = count_reviews_per_movie['count_of_reviews'].mean()

# Making Training Sets

Can build recommendations systems based on:

A. Popularity Recommendation
B. Classification Model
C. Content Recommendation
D. Nearest Neighbor Collaborative Filtering
    - User-based
    - Item-based
E. Association Rule Mining
F. Deep Learning Recommendation
G. Hybrid Approach

Starting with a Popularity Recommendation System

In [158]:
# make training dataset

pop_df = pd.DataFrame(df.groupby('title')['rating'].mean())
pop_df['total number of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
pop_df.head()

,rating,total number of ratings
title,,
'71,4.0,1
'Hellboy': The Seeds of Creation,4.0,1
'Round Midnight,3.5,2
'Salem's Lot,5.0,1
'Til There Was You,4.0,2


In [159]:
# list of 25 top-rated movies
df.groupby('title')['rating'].mean().sort_values(ascending=False).head(25)

title
Dr. Goldfoot and the Bikini Machine                                      5.0
Human Condition III, The (Ningen no joken III)                           5.0
Dylan Moran: Monster                                                     5.0
Justice League: Doom                                                     5.0
Scooby-Doo Goes Hollywood                                                5.0
Scooby-Doo! Abracadabra-Doo                                              5.0
Karlson Returns                                                          5.0
Scooby-Doo! Curse of the Lake Monster                                    5.0
Scooby-Doo! and the Loch Ness Monster                                    5.0
Tom Segura: Mostly Stories                                               5.0
Wings, Legs and Tails                                                    5.0
Tom Segura: Completely Normal                                            5.0
Scooby-Doo! and the Samurai Sword                                     

In [160]:
# the actual training set we need. 
# collab_dataset for collaborative filtering algos
# content_dataset for content algos

collab_dataset = df[['userid', 'movieid', 'rating']]

content_dataset = df.drop(columns=['title', 'imdbid', 'tmdbid', 'genres', 'extra', 'timestamp'])

In [161]:
content_dataset.head()

,userid,movieid,rating,year,no genres listed,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,1995,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,1995,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,1995,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


In [162]:
# Saving datasets
frames = [collab_dataset, content_dataset]
names = ['collab_dataset', 'content_dataset']
for name, data in zip(names, frames):
    data.to_csv(f'../data/{name}.csv')